In [1]:
from main import load_and_preprocess_data

df = load_and_preprocess_data(1)

/Users/heifen/Documents/studie/machinelearning/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading and preprocessing data...
Class distribution:
is_fraud
0    1289169
1       7506
Name: count, dtype: int64
Fraud percentage: 0.58%


In [2]:
from model import load_lstm_model
from predict import predict_and_analyze
from main import SEQUENCE_LENGTH

sample_cc_num = df[df['is_fraud'] == 1]['cc_num'].iloc[0]
user_transactions = df[df['cc_num'] == sample_cc_num]

user_non_fraud_transactions = user_transactions[user_transactions['is_fraud'] == 0].iloc[:-1]
user_fraud_transactions = user_transactions[user_transactions['is_fraud'] == 1]

last_transaction = user_transactions.iloc[-1].to_dict()

lstm_model, processor = load_lstm_model()

results = predict_and_analyze(
    lstm_model,
    user_non_fraud_transactions,
    SEQUENCE_LENGTH,
    processor,
    actual_transaction=last_transaction,
    fraud_transactions=user_fraud_transactions
)

print("\nPredicted most likely values for next transaction:")
print("---------------------------------------------")
for feature, value in results['prediction']['most_likely_values'].items():
    print(f"  {feature}: {value}")
            

print('\nReal values:')
print("---------------------------------------------")
for feature in results['prediction']['most_likely_values'].keys():
    print(f"  {feature}: {last_transaction[feature]}")


Loading model from models/lstm_transaction_model.h5...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step

Predicted most likely values for next transaction:
---------------------------------------------
  amt: {'lower_bound': np.float64(6.17), 'upper_bound': np.float64(6.93), 'avg': np.float64(6.55)}
  lat: {'lower_bound': np.float64(42.6315), 'upper_bound': np.float64(43.033), 'avg': np.float64(42.83225)}
  long: {'lower_bound': np.float64(-112.4777), 'upper_bound': np.float64(-105.6933), 'avg': np.float64(-109.0855)}
  merch_lat: {'lower_bound': np.float64(42.71336933333333), 'upper_bound': np.float64(43.385511466666664), 'avg': np.float64(43.049440399999995)}
  merch_long: {'lower_bound': np.float64(-111.96667829999998), 'upper_bound': np.float64(-105.2959852), 'avg': np.float64(-108.63133174999999)}
  hour: {'avg': np.int64(12)}
  day: {'avg': np.int64(21)}
  month: {'avg': np.int64(6)}
  dayofweek: {'avg': np.int64(5)}
  merchant_encoded: {'avg': np.int64(508)}
  category_encoded: {'avg': np

In [8]:
user_fraud_transactions

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,day,month,dayofweek,age,merchant_encoded,category_encoded,gender_encoded,city_encoded,state_encoded,job_encoded
1017,1017,2019-01-01 12:47:15,60416207185,"fraud_Jones, Sawayn and Romaguera",misc_net,7.27,Mary,Diaz,F,9886 Anita Drive,...,1,1,1,39,293,8,0,272,50,246
2724,2724,2019-01-02 08:44:57,60416207185,fraud_Berge LLC,gas_transport,52.94,Mary,Diaz,F,9886 Anita Drive,...,2,1,2,39,43,2,0,272,50,246
2726,2726,2019-01-02 08:47:36,60416207185,fraud_Luettgen PLC,gas_transport,82.08,Mary,Diaz,F,9886 Anita Drive,...,2,1,2,39,399,2,0,272,50,246
2882,2882,2019-01-02 12:38:14,60416207185,fraud_Daugherty LLC,kids_pets,34.79,Mary,Diaz,F,9886 Anita Drive,...,2,1,2,39,126,7,0,272,50,246
2907,2907,2019-01-02 13:10:46,60416207185,fraud_Beier and Sons,home,27.18,Mary,Diaz,F,9886 Anita Drive,...,2,1,2,39,41,6,0,272,50,246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293561,1293561,2020-06-20 07:56:04,60416207185,fraud_Rutherford-Mertz,grocery_pos,64.78,Mary,Diaz,F,9886 Anita Drive,...,20,6,5,39,543,4,0,272,50,246
1293580,1293580,2020-06-20 08:14:58,60416207185,"fraud_Greenholt, Jacobi and Gleason",gas_transport,66.20,Mary,Diaz,F,9886 Anita Drive,...,20,6,5,39,208,2,0,272,50,246
1294228,1294228,2020-06-20 15:14:35,60416207185,fraud_Stoltenberg-Beatty,shopping_pos,3.74,Mary,Diaz,F,9886 Anita Drive,...,20,6,5,39,601,12,0,272,50,246
1294829,1294829,2020-06-20 20:18:04,60416207185,"fraud_Towne, Greenholt and Koepp",shopping_net,9.42,Mary,Diaz,F,9886 Anita Drive,...,20,6,5,39,630,11,0,272,50,246
